## Set Date Range

In [116]:
start_dt = '2025-11-01'
end_dt = '2025-11-07'

## Invited Users - Batch 1

In [69]:
# invited = pd.read_csv("dw_invited_users.csv") # This is local csv file
invited_users = 'sparky_panel_updated' # This is how data is saved in hive and BQ

## Load Libraries

In [70]:
import os
import pickle
from datetime import datetime, timedelta
from io import BytesIO
from trino.dbapi import connect
from trino.auth import BasicAuthentication

import pandas as pd
import db_dtypes
import pytz
#from src.config import SETTINGS
from google.auth.exceptions import DefaultCredentialsError
from google.cloud import bigquery#, storage
import warnings

warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv()

True

## Setting up BQ and Hive connection

In [71]:
host_name = "presto-datadiscovery.walmart.com"
port_number = 8443
user_name = "k0t0e62"
password = os.getenv("DB_PASSWORD")

In [72]:
conn = connect(
    host=host_name,
    port=port_number,
    user=user_name,
    auth=BasicAuthentication(user_name, password),
    catalog="hive",
    http_scheme="https",
    verify='./merged_cacert.pem'
)

cur = conn.cursor()

In [73]:
def get_bigquery_client(project: str) -> bigquery.Client:
    
    if not os.environ["GOOGLE_APPLICATION_CREDENTIALS"]:
        raise DefaultCredentialsError(
            """GOOGLE_APPLICATION_CREDENTIALS environment variable not set. 
                    Please go to .env file and set it to the path of your GCP service account key file."""
        )

    if not project:
        raise EnvironmentError(
            "GCP_PROJECT environment variable not set. Please go to .env file and set it to your GCP project ID."
        )

    return bigquery.Client(project=os.environ["GCP_PROJECT"])


# ----------------------- Load BigQuery Data -----------------------
def load_bq_data(query: str) -> pd.DataFrame:
    """
    Load data from BigQuery.

    Parameters:
    - query: SQL query to execute
    - client: BigQuery client

    Returns:
    - Pandas DataFrame containing the query result
    """

    client = get_bigquery_client(project=os.environ["GCP_PROJECT"])
    return client.query(query).to_dataframe()


# ----------------------- Load BigQuery Data -----------------------
def write_dataframe_to_bigquery(
    dataframe: pd.DataFrame,
    table_name: str,
    schema=None,
    drop_table=True,
) -> None:
    """
    Write a pandas DataFrame to a BigQuery table.
    ! by default, it drops wmt-transplan-ib-forecasting.transplan_ob_forecasting.{table_name} table
    ! please refactor in the future release to make it more generic

    Parameters:
    - dataframe: Pandas DataFrame to write to BigQuery
    - table_name: Name of the BigQuery table to write to
    - schema: Schema of data structure
    - bq_client: BigQuery client object

    Returns:
    - BigQuery job result
    """
    bq_client = get_bigquery_client(project=os.environ["GCP_PROJECT"])

    if drop_table:
        bq_client.query(
            f"DROP TABLE IF EXISTS wmt-transplan-ib-forecasting.transplan_ib_forecasting.{table_name}"
        )
    target_dataset = bq_client.dataset("transplan_ib_forecasting")
    target_table = target_dataset.table(table_name)
    job = bq_client.load_table_from_dataframe(
        dataframe,
        target_table,
        job_config=schema,
    )
    return job.result()

# Customer Usage Metric 0 - Total classic app sessions

In [74]:
print("Total classic app sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})


SELECT COUNT(DISTINCT a.pg_cust_id) classic_users, 
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) classic_sessn
FROM WW_CSD_DL_TABLES.CSD_SITE_TRAFFIC_EVENT_LOG_VW a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND TENANT_STE_CD In ('US_GLASS_ANDROID', 'US_GLASS_IOS')
AND actn_catg_nm = 'pageView'
AND BOT_IND = 0
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
glass_app_session = pd.DataFrame(rows, columns=cols)
glass_app_session.head()



Total classic app sessions


,classic_users,classic_sessn
0,12240,108805


## Customer Usage Engagement Funnel Metric 1 - Logged In Users/ Sessions

In [75]:
print("Logged in users/ sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT COUNT(DISTINCT a.pg_cust_id) AS login_users,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS login_sessn
FROM (SELECT DISTINCT pg_cust_id, visitor_cooke_val, brwsr_sessn_cooke_val
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND json_extract_scalar(page_attr, '$.nm') = 'homePage' -- Users that reached home page
AND actn_catg_nm = 'pageView') a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
login_users = pd.DataFrame(rows, columns=cols)
login_users.head()

Logged in users/ sessions


,login_users,login_sessn
0,6085,17965


## Customer Usage Engagement Funnel Metric 2 - Interacted Users and sessions

In [76]:
print("Interacted Users/Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS interacted_users,
  COUNT(DISTINCT brwsr_sessn_cooke_val) AS interacted_sessn
FROM (SELECT DISTINCT pg_cust_id, brwsr_sessn_cooke_val FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND pg_cust_id IS NOT NULL) a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
interacted_users = pd.DataFrame(rows, columns=cols)
interacted_users.head()

Interacted Users/Sessions


,interacted_users,interacted_sessn
0,5901,16634


## Metric 3- Users with at least 1 bounce session/ Bounce Sessions

In [77]:
print("Bounced Users/ Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

bounce_sessions AS (
SELECT
  pg_cust_id,
  brwsr_sessn_cooke_val
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (assoc_ind = 'false' OR assoc_ind IS NULL)
  AND actn_catg_nm IN ('onClick','pageView')
  AND pg_cust_id IS NOT NULL
GROUP BY pg_cust_id, brwsr_sessn_cooke_val
HAVING
  -- no clicks in the session
  SUM(CASE WHEN actn_catg_nm = 'onClick' THEN 1 ELSE 0 END) = 0
  -- at least one homepage pageView
  AND SUM(CASE WHEN actn_catg_nm = 'pageView' AND cntxt_nm = 'homePage' THEN 1 ELSE 0 END) >= 1
  -- no pageViews that are NOT homepage
  AND SUM(CASE WHEN actn_catg_nm = 'pageView' AND cntxt_nm <> 'homePage' THEN 1 ELSE 0 END) = 0)


SELECT COUNT(DISTINCT a.pg_cust_id) bounce_cust_ct,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) bounce_sessn_ct
FROM bounce_sessions a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
bounce_users = pd.DataFrame(rows, columns=cols)
bounce_users.head()

Bounced Users/ Sessions


,bounce_cust_ct,bounce_sessn_ct
0,39,41


## Customer usage Engagement Funnel Metric 
 1. Meaningfully Interacted Users, Session, Count
 2. Users, Sessions, Count with message
 3. Users, Sessions, Count of Click category tile
 4. Users, Sessions, Count of Click Home Page tile
 5. Users, Sessions, Count of Click Recent tile

In [78]:
print("Meaningfully Interacted Users")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

meaningful_interactions AS (SELECT
  pg_cust_id,
  MAX(CASE WHEN (json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') THEN 1 ELSE 0 END) AS sent_msg,
  MAX(CASE WHEN (cntxt_nm = 'recentPage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')) THEN 1 ELSE 0 END) AS c_recent_card,
  MAX(CASE WHEN (cntxt_nm = 'homePage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')) THEN 1 ELSE 0 END) AS c_home_card,
  MAX(CASE WHEN (cntxt_nm = 'categoryPage' AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard') THEN 1 ELSE 0 END) AS c_category_card
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND ((json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') -- Sent a message on Sparky
    OR (cntxt_nm = 'recentPage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
      ) -- Clicked on Recent cards
    OR (cntxt_nm = 'homePage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
      ) -- Clicked on Home Page Cards
    OR (cntxt_nm = 'categoryPage'
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard'
      ) --  Clicked on Category Cards
    )
GROUP BY 1
)


SELECT COUNT(DISTINCT a.pg_cust_id) AS users_meaningful_interaction,
       COUNT(DISTINCT CASE WHEN sent_msg = 1 THEN a.pg_cust_id END) AS users_sent_msg,
       COUNT(DISTINCT CASE WHEN c_recent_card = 1 THEN a.pg_cust_id END) AS users_click_recent_card,
       COUNT(DISTINCT CASE WHEN c_home_card = 1 THEN a.pg_cust_id END) AS users_click_home_card,
       COUNT(DISTINCT CASE WHEN c_category_card = 1 THEN a.pg_cust_id END) AS users_click_category_card
FROM meaningful_interactions a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users

"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
meaningful_interacted_users = pd.DataFrame(rows, columns=cols)
meaningful_interacted_users.head()

Meaningfully Interacted Users


,users_meaningful_interaction,users_sent_msg,users_click_recent_card,users_click_home_card,users_click_category_card
0,5247,4497,1667,1913,2781


In [79]:
print("Meaningfully Interacted Sessions")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

meaningful_interactions AS (SELECT a.pg_cust_id,
  brwsr_sessn_cooke_val,
  MAX(CASE WHEN (json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') THEN 1 ELSE 0 END) AS sent_msg,
  MAX(CASE WHEN (cntxt_nm = 'recentPage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')) THEN 1 ELSE 0 END) AS c_recent_card,
  MAX(CASE WHEN (cntxt_nm = 'homePage' AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')) THEN 1 ELSE 0 END) AS c_home_card,
  MAX(CASE WHEN (cntxt_nm = 'categoryPage' AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard') THEN 1 ELSE 0 END) AS c_category_card
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND ((json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND') -- Sent a message on Sparky
    OR (cntxt_nm = 'recentPage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
      ) -- Clicked on Recent cards
    OR (cntxt_nm = 'homePage' 
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
      ) -- Clicked on Home Page Cards
    OR (cntxt_nm = 'categoryPage'
          AND actn_catg_nm = 'onClick' 
          AND json_extract_scalar(payload_txt, '$.actionSubCateg') = 'cardClick' 
          AND json_extract_scalar(payload_txt, '$.moduleType') = 'categoryCard'
      ) --  Clicked on Category Cards
    )
GROUP BY 1,2
)


SELECT 
      COUNT(DISTINCT brwsr_sessn_cooke_val) AS sessn_meaningful_interaction,
      COUNT(DISTINCT CASE WHEN sent_msg = 1 THEN brwsr_sessn_cooke_val END) AS sessn_sent_msg,
      COUNT(DISTINCT CASE WHEN c_recent_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_recent_card,
      COUNT(DISTINCT CASE WHEN c_home_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_home_card,
      COUNT(DISTINCT CASE WHEN c_category_card = 1 THEN brwsr_sessn_cooke_val END) AS sessn_click_category_card
FROM meaningful_interactions
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
meaningful_interacted_sessn = pd.DataFrame(rows, columns=cols)
meaningful_interacted_sessn.head()

Meaningfully Interacted Sessions


,sessn_meaningful_interaction,sessn_sent_msg,sessn_click_recent_card,sessn_click_home_card,sessn_click_category_card
0,13026,9382,2682,2526,4357


In [80]:
print("Event Counts for Each Interaction Type")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT

    -- ALL meaningful interaction events (union of all above, counted at the event/log level, not user)
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND (
                json_extract_scalar(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
                OR (
                    e.cntxt_nm = 'recentPage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
                )
                OR (
                    e.cntxt_nm = 'homePage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
                )
                OR (
                    e.cntxt_nm = 'categoryPage'
                    AND e.actn_catg_nm = 'onClick'
                    AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
                    AND json_extract_scalar(e.payload_txt, '$.moduleType') = 'categoryCard'
                )
          )
    ) AS count_meaningful_interaction,

    -- Sent message count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND json_extract_scalar(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
    ) AS count_sent_msg,

    -- Clicked recent card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'recentPage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('OrderCard', 'ChatCard', 'ShopCard')
    ) AS count_recent_card,

    -- Clicked home card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'homePage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleName') IN ('SHOP', 'WELCOME', 'PRODUCT')
    ) AS count_home_card,

    -- Clicked category card count
    (SELECT COUNT(*)
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
        JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
        WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
          AND e.cntxt_nm = 'categoryPage'
          AND e.actn_catg_nm = 'onClick'
          AND json_extract_scalar(e.payload_txt, '$.actionSubCateg') = 'cardClick'
          AND json_extract_scalar(e.payload_txt, '$.moduleType') = 'categoryCard'
    ) AS count_category_card

    
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
meaningful_interected_event_counts = pd.DataFrame(rows, columns=cols)
meaningful_interected_event_counts.head()

Event Counts for Each Interaction Type


,count_meaningful_interaction,count_sent_msg,count_recent_card,count_home_card,count_category_card
0,37901,23115,4352,3590,6844


## Customer usage Engagement Funnel Metric 
 1. Users, Session, Count who open shop
 2. Users, Sessions, Count who open category shop
 3. Users, Sessions, Count who open weekly shop
 4. Users, Sessions, Count who open browse shop
 5. Users, Sessions, Count who open Planning (events) shop 

In [81]:
print("Users that opened a shop")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_opened_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'eventShop' THEN a.pg_cust_id END) AS cust_opened_event_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'weeklyShop' THEN a.pg_cust_id END) AS cust_opened_weekly_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'browseShop' THEN a.pg_cust_id END) AS cust_opened_browse_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'categoryShop' THEN a.pg_cust_id END) AS cust_opened_category_shop
FROM (SELECT DISTINCT pg_cust_id, cntxt_nm FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'pageView'
AND cntxt_nm in ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')) a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
opened_shop_cust = pd.DataFrame(rows, columns=cols)
opened_shop_cust.head()

Users that opened a shop


,cust_opened_shop,cust_opened_event_shop,cust_opened_weekly_shop,cust_opened_browse_shop,cust_opened_category_shop
0,3321,194,885,87,2946


In [82]:
print("Sessions in which a shop was open")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_opened_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'eventShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_event_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'weeklyShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_weekly_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'browseShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_browse_shop,
  COUNT(DISTINCT CASE WHEN cntxt_nm = 'categoryShop' THEN a.brwsr_sessn_cooke_val END) AS sessn_opened_category_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'pageView'
AND cntxt_nm in ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
opened_shop_sessn = pd.DataFrame(rows, columns=cols)
opened_shop_sessn.head()

Sessions in which a shop was open


,sessn_opened_shop,sessn_opened_event_shop,sessn_opened_weekly_shop,sessn_opened_browse_shop,sessn_opened_category_shop
0,5448,230,1086,100,4696


In [83]:
print("Count/Number of times a Shop Opened")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),

impressions AS (
    SELECT *,
        json_extract_scalar(payload_txt, '$.pageViewId') AS pageViewId,
        json_extract_scalar(page_attr, '$.nm') AS page_nm
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
    JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
    WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
      AND e.actn_catg_nm = 'pageView'
      ---AND e.cntxt_nm IN ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop', 'cart', 'orderDetails', 'discoverMore')
)

SELECT
    COUNT(DISTINCT CASE 
        WHEN cntxt_nm IN ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop')
        THEN pageViewId END) AS count_opened_shop,
    
    COUNT(DISTINCT CASE 
        WHEN cntxt_nm = 'eventShop' AND page_nm = 'eventShop'
        THEN pageViewId END) AS count_opened_event_shop,

    COUNT(DISTINCT CASE 
        WHEN cntxt_nm = 'weeklyShop' AND page_nm = 'weeklyShop'
        THEN pageViewId END) AS count_opened_weekly_shop,

    COUNT(DISTINCT CASE 
        WHEN cntxt_nm = 'browseShop' AND page_nm = 'browseShop'
        THEN pageViewId END) AS count_opened_browse_shop,

    COUNT(DISTINCT CASE 
        WHEN cntxt_nm = 'categoryShop' AND page_nm = 'categoryShop'
        THEN pageViewId END) AS count_opened_category_shop

FROM impressions
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
count_opened_shop = pd.DataFrame(rows, columns=cols)
count_opened_shop.head()


Count/Number of times a Shop Opened


,count_opened_shop,count_opened_event_shop,count_opened_weekly_shop,count_opened_browse_shop,count_opened_category_shop
0,16086,415,2573,245,12853


## Customer usage Engagement Funnel Metric 
 1. Users, Count that created shop
 2. Users, Count that created browse shop
 3. Users, Count that created Planning (events) shop 

In [84]:
print("Users that created a shop")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT
    COUNT(DISTINCT e.pg_cust_id) AS created_shop_users,
    COUNT(DISTINCT e.brwsr_sessn_cooke_val) AS created_shop_sessn,
    COUNT(*) AS count_created_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_RECEIVE'
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.responseType') = 'SHOP_MESSAGE'
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
created_shop = pd.DataFrame(rows, columns=cols)
created_shop.head()




Users that created a shop


,created_shop_users,created_shop_sessn,count_created_shop
0,237,260,321


In [85]:
print("Users that created a shop")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
)
SELECT
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN e.pg_cust_id END) AS users_created_event_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN e.pg_cust_id END) AS users_created_browse_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN brwsr_sessn_cooke_val END) AS sessn_created_event_shop,
    COUNT(DISTINCT CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN brwsr_sessn_cooke_val END) AS sessn_created_browse_shop,
    COUNT(CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'EVENT' THEN 1 END) AS count_created_event_shop,
    COUNT(CASE WHEN json_extract_scalar(payload_txt, '$.shopType') = 'BROWSE' THEN 1 END) AS count_created_browse_shop
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
  AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.eventType') = 'SPARKY_CHAT_RECEIVE'
  AND JSON_EXTRACT_SCALAR(e.payload_txt, '$.responseType') = 'SHOP_MESSAGE'
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
created_shop_type = pd.DataFrame(rows, columns=cols)
created_shop_type.head()


Users that created a shop


,users_created_event_shop,users_created_browse_shop,sessn_created_event_shop,sessn_created_browse_shop,count_created_event_shop,count_created_browse_shop
0,152,78,164,82,193,108


## Customer usage Engagement Funnel Metric - Users, Session, Count with ATC / Add All to cart


In [86]:
print("Users that did ATC/ Total ATCs")
sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

atc_clicks as (SELECT event_dt, 
       pg_cust_id, 
       brwsr_sessn_cooke_val,
       actn_catg_nm,
       actn_sub_catg_nm,
       cntxt_nm,
       CAST(from_unixtime(cast(event_ts_epoch as double) / 1000) AT TIME ZONE 'America/Los_Angeles' AS timestamp) AS event_ts_epoch,
       json_extract_scalar(page_attr, '$.nm') AS page_nm,
       json_extract_scalar(payload_txt, '$.r') AS ref_page,
       json_extract_scalar(payload_txt, '$.overlayName') AS overlay_nm,
       COALESCE(json_extract_scalar(payload_txt, '$.itemDetails.itemId'), 
                json_extract_scalar(payload_txt, '$.itemId'),
                json_extract_scalar(payload_txt, '$.itemDetails.usitemId')) AS item_id,
       MAX(json_extract_scalar(payload_txt, '$.shopId')) shop_id
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'onClick'
AND actn_sub_catg_nm in ('addToCart', 'addAllToCart')
AND json_extract_scalar(page_attr, '$.nm') != 'arPlatform'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11),

dedupe AS (SELECT a.*, 
          row_number() OVER (PARTITION BY a.pg_cust_id, brwsr_sessn_cooke_val, actn_sub_catg_nm, item_id ORDER BY shop_id DESC, overlay_nm DESC, event_ts_epoch) AS rn_atc, -- Remove duplicates add to carts
          row_number() OVER (PARTITION BY a.pg_cust_id, brwsr_sessn_cooke_val, actn_sub_catg_nm, cntxt_nm, shop_id, item_id ORDER BY event_ts_epoch) AS rn_aatc -- Remove duplicates add all to carts
FROM atc_clicks a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id)

SELECT 
    COUNT(DISTINCT a.pg_cust_id) AS cust_w_atc_both,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN a.pg_cust_id END) AS cust_w_aatc,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addToCart' THEN a.pg_cust_id END) AS cust_w_atc,
    COUNT(DISTINCT brwsr_sessn_cooke_val) AS sessn_w_atc_both,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN a.brwsr_sessn_cooke_val END) AS sessn_w_aatc,
    COUNT(DISTINCT CASE WHEN actn_sub_catg_nm = 'addToCart' THEN a.brwsr_sessn_cooke_val END) AS sessn_w_atc,
    COUNT(*) AS total_atc,
    COUNT(CASE WHEN actn_sub_catg_nm = 'addToCart' THEN 1 END) AS count_w_atc,
    COUNT(CASE WHEN actn_sub_catg_nm = 'addAllToCart' THEN 1 END) AS count_w_aatc
FROM dedupe a
WHERE CASE WHEN actn_sub_catg_nm = 'addToCart' THEN rn_atc = 1 ELSE rn_aatc = 1 END
"""
cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]  # extract column names
atc_users = pd.DataFrame(rows, columns=cols)
atc_users.head()

Users that did ATC/ Total ATCs


,cust_w_atc_both,cust_w_aatc,cust_w_atc,sessn_w_atc_both,sessn_w_aatc,sessn_w_atc,total_atc,count_w_atc,count_w_aatc
0,1935,159,1906,3405,167,3350,11858,11690,168


## Users and sessions that clicked on continue to checkout


In [87]:
print("Users that clicked on continue to cart")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_w_checkout
FROM (SELECT DISTINCT pg_cust_id FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND actn_sub_catg_nm = 'continueToCheckout'
AND cntxt_nm = 'cart') a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users

"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
checkout_users = pd.DataFrame(rows, columns=cols)
checkout_users.head()

Users that clicked on continue to cart


,cust_w_checkout
0,909


In [88]:
print("Sessions with click on continue to checkout")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_w_checkout
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm IN ('onClick') 
AND actn_sub_catg_nm = 'continueToCheckout'
AND cntxt_nm = 'cart'-- This will exclude the onboarding events
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
checkout_sessn = pd.DataFrame(rows, columns=cols)
checkout_sessn.head()

Sessions with click on continue to checkout


,sessn_w_checkout
0,1190


## Customer Usage Engagement Funnel - Users, Sessions that had an order / Total orders


In [89]:
print("Users that placed an order/ Total orders")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.pg_cust_id) AS cust_w_orders,
  COUNT(DISTINCT order_id) as total_orders
FROM (SELECT DISTINCT event_dt,
                 pg_cust_id, 
                 json_extract_scalar(payload_txt, '$.orderId') as order_id
      FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
      WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
      AND cntxt_nm = 'checkout'
      AND json_extract_scalar(page_attr, '$.nm') = 'thankYou'
      AND actn_catg_nm = 'pageView') a -- This will exclude the onboarding events
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
order_users = pd.DataFrame(rows, columns=cols)
order_users.head()

Users that placed an order/ Total orders


,cust_w_orders,total_orders
0,658,853


In [90]:
print("Sessions with orders")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users})

SELECT
  COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS sessn_w_orders
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a 
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND cntxt_nm = 'checkout'
AND json_extract_scalar(page_attr, '$.nm') = 'thankYou'
AND actn_catg_nm = 'pageView' -- This will exclude the onboarding events
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
order_sessn = pd.DataFrame(rows, columns=cols)
order_sessn.head()

Sessions with orders


,sessn_w_orders
0,831


## Customer Usage Engagement Funnel - Customers with repeated orders.

In [117]:
print("Customers with repeated orders (more than one order)")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),
orders_by_user AS (
    SELECT
        e.pg_cust_id,
        COUNT(DISTINCT JSON_EXTRACT_SCALAR(e.payload_txt, '$.orderId')) AS total_orders
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
    JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
    WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
      AND e.cntxt_nm = 'checkout'
      AND JSON_EXTRACT_SCALAR(e.page_attr, '$.nm') = 'thankYou'
      AND e.actn_catg_nm = 'pageView'
    GROUP BY e.pg_cust_id
)
SELECT
    COUNT(Distinct pg_cust_id) AS users_with_repeated_orders
FROM orders_by_user
WHERE total_orders > 1
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
repeated_order_users = pd.DataFrame(rows, columns=cols)
repeated_order_users.head()


Customers with repeated orders (more than one order)


,users_with_repeated_orders
0,137


## Metric - In Chat Bounced Users


In [91]:
print("In Chat Bounced Users")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

clicked_on_sparky AS (
    SELECT DISTINCT 
        pg_cust_id, 
        brwsr_sessn_cooke_val
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
    AND actn_catg_nm = 'onClick'
    AND actn_sub_catg_nm = 'sparky'
),

sent_msg AS (
    SELECT DISTINCT 
        pg_cust_id, 
        brwsr_sessn_cooke_val
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
    AND json_extract_scalar(payload_txt, '$.eventType') = 'SPARKY_CHAT_SEND'
)


SELECT COUNT(DISTINCT a.pg_cust_id) AS chat_bounce_users,
       COUNT(DISTINCT a.brwsr_sessn_cooke_val) AS chat_bounce_sessn
FROM clicked_on_sparky a
LEFT JOIN sent_msg b 
ON a.pg_cust_id = b.pg_cust_id AND a.brwsr_sessn_cooke_val = b.brwsr_sessn_cooke_val
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id -- To only keep the invited users
WHERE b.brwsr_sessn_cooke_val is null
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]   # extract column names
inchat_bounce_users = pd.DataFrame(rows, columns=cols)
inchat_bounce_users.head()

In Chat Bounced Users


,chat_bounce_users,chat_bounce_sessn
0,556,590


## System Performance - Total Messages, Error, Fallback

In [92]:
print("Total Messages, Error, Fallback")

inchat_msg = load_bq_data(f"""
                   WITH invited_users AS (
                    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
                    FROM `wmt-et-ccomm-prod.temp.{invited_users}`)

                  SELECT
                  COUNT(DISTINCT correlation_id) AS total_msg,
                  COUNT(DISTINCT CASE WHEN REGEXP_EXTRACT(current_entities, '\\\\{{([^=]+)=' ) = 'fallback_category' THEN correlation_id END) AS fallback_msg,
                  COUNT(DISTINCT CASE WHEN converse_intent is null
                                      OR converse_intent = ""
                                      OR converse_short_reply like "%Sorry, there was a problem while processing your request. Can I help you with anything else?%"
                                      OR converse_short_reply like "%Your user id is already in use. Please try again after sometime.%"
                                      OR converse_short_reply like "%not quite working on my end, please try again later.%"
                                      THEN correlation_id END) AS error_msg
                  FROM `wmt-et-converse-prod.converse_logs_ds_xrsamurai.logs_parsed` a
                  JOIN invited_users i
                  ON i.pg_cust_id = a.customer_id -- To only keep the invited users
                  WHERE DATE(DATETIME(TIMESTAMP(timestamp), 'America/Los_Angeles')) BETWEEN '{start_dt}' AND '{end_dt}'
                  AND channel_id = 'RAPTOR_IOS_APP'
                    """)

inchat_msg.head()


Total Messages, Error, Fallback


,total_msg,fallback_msg,error_msg
0,22075,1012,572


## System Performance - Latency (p50, p75, p90, p95, p99)

In [93]:
print("Latency Percentiles (p50, p75, p90, p95, p99)")

latency_percentiles = load_bq_data(f"""
    WITH invited_users AS (
        SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
        FROM `wmt-et-ccomm-prod.temp.{invited_users}`
    ),
    ml_json AS (
        SELECT
            correlation_id,
            CASE
                WHEN REGEXP_CONTAINS(customMetadata, r'"module_latency":"\\{{') THEN
                    REPLACE(JSON_EXTRACT_SCALAR(customMetadata, '$.module_latency'), '\\\\', '')
                ELSE
                    JSON_EXTRACT(customMetadata, '$.module_latency')
            END AS ml_clean
        FROM `wmt-et-converse-prod.converse_logs_ds_xrsamurai.logs_parsed` a
        JOIN invited_users i ON i.pg_cust_id = a.customer_id
        WHERE DATE(DATETIME(TIMESTAMP(a.timestamp), 'America/Los_Angeles')) BETWEEN '{start_dt}' AND '{end_dt}'
            AND a.channel_id = 'RAPTOR_IOS_APP'
    ),
    latency AS (
        SELECT
            correlation_id,
            IFNULL(
                SUM(CAST(REGEXP_EXTRACT(pair, r'"[^"]+":"(\\d+)"') AS INT64)), 0
            )/1000.0 AS total_latency_sec
        FROM ml_json,
            UNNEST(REGEXP_EXTRACT_ALL(ml_clean, r'"[^"]+":"\\d+"')) AS pair
        WHERE
            ml_clean IS NOT NULL
            AND ml_clean != '{{}}'
            AND REGEXP_EXTRACT(pair, r'"([^"]+)":') != 'xrsamurai:CortexService'
        GROUP BY correlation_id
    )
    SELECT
        APPROX_QUANTILES(total_latency_sec, 100)[OFFSET(50)] AS p50_latency,
        APPROX_QUANTILES(total_latency_sec, 100)[OFFSET(75)] AS p75_latency,
        APPROX_QUANTILES(total_latency_sec, 100)[OFFSET(90)] AS p90_latency,
        APPROX_QUANTILES(total_latency_sec, 100)[OFFSET(95)] AS p95_latency,
        APPROX_QUANTILES(total_latency_sec, 100)[OFFSET(99)] AS p99_latency
    FROM latency
""")

latency_percentiles.head()


Latency Percentiles (p50, p75, p90, p95, p99)


,p50_latency,p75_latency,p90_latency,p95_latency,p99_latency
0,8.249,9.475,10.781,11.773,22.682


## Interaction Experience - Duration of Session

In [94]:
print("Average, Min, and Max Session Duration (ms and seconds)")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),
session_durations AS (
    SELECT
        brwsr_sessn_cooke_val AS session_id,
        CAST(MIN(CAST(event_ts_epoch AS BIGINT)) AS BIGINT) AS min_ts,
        CAST(MAX(CAST(event_ts_epoch AS BIGINT)) AS BIGINT) AS max_ts,
        MAX(CAST(event_ts_epoch AS BIGINT)) - MIN(CAST(event_ts_epoch AS BIGINT)) AS session_duration_ms
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
    JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
    WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    GROUP BY brwsr_sessn_cooke_val
)
SELECT
    ROUND(AVG(session_duration_ms) / 1000, 2) AS avg_session_duration_sec,
    ROUND(MIN(session_duration_ms) / 1000, 2) AS min_session_duration_sec,
    ROUND(MAX(session_duration_ms) / 1000, 2) AS max_session_duration_sec
FROM session_durations
WHERE session_duration_ms > 0
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
session_duration = pd.DataFrame(rows, columns=cols)
session_duration.head()


Average, Min, and Max Session Duration (ms and seconds)


,avg_session_duration_sec,min_session_duration_sec,max_session_duration_sec
0,302.17,0,19671


## Users with Carousel Impressions, Total Carousel Impressions and Total Item Impressions

In [95]:
print("Users with item carousel impressions")


users_w_carousel_imp = load_bq_data(f"""
WITH invited_users AS (
 SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
 FROM `wmt-et-ccomm-prod.temp.{invited_users}`),

carousel AS (SELECT 
*,
REGEXP_EXTRACT(current_entities, r'products=\[([\s\S]*?)\]') AS products_inner
FROM `wmt-et-converse-prod.converse_logs_ds_xrsamurai.logs_parsed` a
JOIN invited_users i
ON i.pg_cust_id = a.customer_id -- To only keep the invited users
WHERE DATE(DATETIME(TIMESTAMP(timestamp), 'America/Los_Angeles')) BETWEEN '{start_dt}' AND '{end_dt}'
AND channel_id = 'RAPTOR_IOS_APP'),

carousel_item AS (SELECT
 customer_id,
 correlation_id,
 ARRAY_AGG(JSON_EXTRACT_SCALAR(product, '$.usItemId')) AS usItemIds,
 ARRAY_LENGTH(ARRAY_AGG(JSON_EXTRACT_SCALAR(product, '$.usItemId'))) AS usItemCount
FROM carousel,
UNNEST(JSON_EXTRACT_ARRAY(CONCAT('[', products_inner, ']'), '$')) AS product
WHERE products_inner IS NOT NULL
GROUP BY 1, 2)

SELECT COUNT(DISTINCT customer_id) users_w_carousel_imp, Count(Distinct correlation_id) as carousel_imp, SUM(usItemCount) total_item_imp
FROM carousel_item
""")

users_w_carousel_imp.head()

Users with item carousel impressions


,users_w_carousel_imp,carousel_imp,total_item_imp
0,3958,17082,81463


## Shop Engagement Funnel by Volume - PDP page Impressions

In [96]:
print("Total Product Page Inmpressions")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),

impressions AS (
    SELECT *,
        json_extract_scalar(payload_txt, '$.pageViewId') AS pageViewId,
        json_extract_scalar(page_attr, '$.nm') AS page_nm
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw e
    JOIN invited_users iu ON iu.pg_cust_id = e.pg_cust_id
    WHERE e.event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (e.assoc_ind = 'false' OR e.assoc_ind IS NULL)
      AND e.actn_catg_nm = 'pageView'
      ---AND e.cntxt_nm IN ('eventShop', 'weeklyShop', 'browseShop', 'categoryShop', 'cart', 'orderDetails', 'discoverMore')
)

SELECT
    COUNT(DISTINCT CASE 
        WHEN (cntxt_nm = 'productPage' AND page_nm = 'productPage')
        THEN pageViewId END) AS productPage_impressions
FROM impressions
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
pdp_imp = pd.DataFrame(rows, columns=cols)
pdp_imp.head()

Total Product Page Inmpressions


,productPage_impressions
0,19893


## Cutomer Usage - Shop Engagement Volume - ATCs

In [97]:
print("ATC Context Category Breakdown")

sql = f"""
WITH invited_users AS (
SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
FROM gpadata_analytics.{invited_users}),

atc_clicks as (SELECT event_dt, 
       pg_cust_id, 
       brwsr_sessn_cooke_val,
       actn_catg_nm,
       actn_sub_catg_nm,
       cntxt_nm,
       CAST(from_unixtime(cast(event_ts_epoch as double) / 1000) AT TIME ZONE 'America/Los_Angeles' AS timestamp) AS event_ts_epoch,
       json_extract_scalar(page_attr, '$.nm') AS page_nm,
       json_extract_scalar(payload_txt, '$.r') AS referencepage,
       json_extract_scalar(payload_txt, '$.overlayName') AS overlay_nm,
       COALESCE(json_extract_scalar(payload_txt, '$.itemDetails.itemId'), 
                json_extract_scalar(payload_txt, '$.itemId'),
                json_extract_scalar(payload_txt, '$.itemDetails.usitemId')) AS item_id,
       MAX(json_extract_scalar(payload_txt, '$.shopId')) shop_id
FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
AND (assoc_ind = 'false' OR assoc_ind is null) -- Exclude the associates
AND actn_catg_nm = 'onClick'
AND actn_sub_catg_nm in ('addToCart', 'addAllToCart')
AND json_extract_scalar(page_attr, '$.nm') != 'arPlatform'
GROUP BY 1,2,3,4,5,6,7,8,9,10,11),

dedupe AS (SELECT a.*, 
          row_number() OVER (PARTITION BY a.pg_cust_id, brwsr_sessn_cooke_val, actn_sub_catg_nm, item_id ORDER BY shop_id DESC, overlay_nm DESC, event_ts_epoch) AS rn_atc, -- Remove duplicates add to carts
          row_number() OVER (PARTITION BY a.pg_cust_id, brwsr_sessn_cooke_val, actn_sub_catg_nm, cntxt_nm, shop_id, item_id ORDER BY event_ts_epoch) AS rn_aatc -- Remove duplicates add all to carts
FROM atc_clicks a
JOIN invited_users i
ON i.pg_cust_id = a.pg_cust_id),

atc AS (
    SELECT
        event_dt,
        pg_cust_id,
        brwsr_sessn_cooke_val,
        actn_catg_nm,
        actn_sub_catg_nm,
        cntxt_nm,
        event_ts_epoch,
        page_nm,
        referencepage,
        overlay_nm,
        item_id,
        shop_id
    FROM dedupe
    WHERE CASE WHEN actn_sub_catg_nm = 'addToCart' THEN rn_atc = 1 ELSE rn_aatc = 1 END
)

SELECT
    CASE
        WHEN cntxt_nm = 'eventShop' AND page_nm = 'eventShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectEventShopATC'
        WHEN cntxt_nm = 'eventShop' AND page_nm = 'eventShop' AND actn_sub_catg_nm = 'addAllToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectEventShopAATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'eventShop' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPEventShopATC'
        WHEN cntxt_nm = 'weeklyShop' AND page_nm = 'weeklyShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectWeeklyShopATC'
        WHEN cntxt_nm = 'weeklyShop' AND page_nm = 'weeklyShop' AND actn_sub_catg_nm = 'addAllToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectWeeklyShopAATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'weeklyShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'PDPWeeklyShopATC'
        WHEN cntxt_nm = 'browseShop' AND page_nm = 'browseShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectBrowseShopATC'
        WHEN cntxt_nm = 'browseShop' AND page_nm = 'browseShop' AND actn_sub_catg_nm = 'addAllToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectBrowseShopAATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'browseShop' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPBrowseShopATC'
        WHEN cntxt_nm = 'categoryShop' AND page_nm = 'categoryShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectCategoryShopATC'
        WHEN cntxt_nm = 'categoryShop' AND page_nm = 'categoryShop' AND actn_sub_catg_nm = 'addAllToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectCategoryShopAATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'categoryShop' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'PDPCategoryShopATC'
        WHEN ((cntxt_nm = 'sparkyAssistant' AND page_nm = 'productPage') OR (overlay_nm = 'sparkyChatWindow')) THEN 'DirectCarouselATC'
        WHEN (cntxt_nm = 'shopPage' AND page_nm = 'discoverMore' AND actn_sub_catg_nm in ('addToCart')) THEN 'DiscoverMoreATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'discoverMore' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPDiscoverMoreATC'
        WHEN cntxt_nm = 'cart' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectCartATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage IN ('cart') AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPCartATC'
        WHEN cntxt_nm = 'orderDetails' AND page_nm = 'orderDetails' AND actn_sub_catg_nm = 'addToCart' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' THEN 'DirectOrderDetailsATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'orderDetails' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPOrderDetailsATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND referencepage = 'productPage' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' AND actn_sub_catg_nm = 'addToCart' THEN 'PDPProductPageATC'
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage' AND COALESCE(overlay_nm,'') <> 'sparkyChatWindow' AND actn_sub_catg_nm = 'addToCart' THEN 'OverallProductPageATC'
        ELSE 'others'
    END AS context_category,
    COUNT(*) as volume
FROM atc
GROUP BY 1
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
atc_context = pd.DataFrame(rows, columns=cols)
# 1. Melt the DataFrame to stack reach and volume
melted = atc_context.melt(id_vars='context_category', var_name='metric', value_name='value')
# 2. Create a new column for combined names
melted['new_col'] = melted['context_category'] + '_' + melted['metric']
# 3. Pivot to a single row
pivoted = melted.pivot_table(index=[], columns='new_col', values='value')
# 4. If you want a flat DataFrame (not MultiIndex columns)
pivoted.columns.name = None  # Remove the columns name
atc_context_pivot = pivoted.reset_index(drop=True)  # Reset index, if necessary
atc_context_pivot

ATC Context Category Breakdown


,DirectBrowseShopATC_volume,DirectCarouselATC_volume,DirectCartATC_volume,DirectCategoryShopATC_volume,DirectEventShopAATC_volume,DirectEventShopATC_volume,DirectOrderDetailsATC_volume,DirectWeeklyShopAATC_volume,DirectWeeklyShopATC_volume,DiscoverMoreATC_volume,OverallProductPageATC_volume,PDPBrowseShopATC_volume,PDPCartATC_volume,PDPCategoryShopATC_volume,PDPDiscoverMoreATC_volume,PDPEventShopATC_volume,PDPOrderDetailsATC_volume,PDPProductPageATC_volume,PDPWeeklyShopATC_volume,others_volume
0,79.0,5561.0,1600.0,786.0,21.0,18.0,699.0,147.0,263.0,525.0,388.0,5.0,98.0,286.0,290.0,1.0,31.0,1013.0,20.0,27.0


## Customer Usage - Shop Engagement Volume - Item Clicks

In [98]:
print("Item Clicks Grouped by Context Category")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),

item_clicks AS (
    SELECT DISTINCT
        event_dt,
        a.pg_cust_id,
        brwsr_sessn_cooke_val,
        actn_catg_nm,
        actn_sub_catg_nm,
        cntxt_nm,
        CAST(
            from_unixtime(CAST(event_ts_epoch AS double) / 1000)
            AT TIME ZONE 'America/Los_Angeles' AS timestamp
        ) AS event_ts_epoch,
        json_extract_scalar(payload_txt, '$.pageViewId') AS pageViewId,
        json_extract_scalar(page_attr, '$.nm') AS page_nm,
        json_extract_scalar(payload_txt, '$.r') AS ref_page,
        json_extract_scalar(payload_txt, '$.overlayName') AS overlay_nm,
        COALESCE(
            json_extract_scalar(payload_txt, '$.itemDetails.itemId'),
            json_extract_scalar(payload_txt, '$.itemId'),
            json_extract_scalar(payload_txt, '$.itemDetails.usitemId')
        ) AS item_id,
        json_extract_scalar(payload_txt, '$.shopId') AS shop_id
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a
    JOIN invited_users i ON i.pg_cust_id = a.pg_cust_id
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (assoc_ind = 'false' OR assoc_ind IS NULL)
      AND actn_sub_catg_nm = 'itemClick'
)

SELECT
    SUM(CASE 
        WHEN cntxt_nm = 'eventShop' AND page_nm = 'eventShop' AND overlay_nm IS NULL
        THEN 1 ELSE 0 END) AS event_shop_itemClick,

    SUM(CASE 
        WHEN cntxt_nm = 'weeklyShop' AND page_nm = 'weeklyShop' AND overlay_nm IS NULL
        THEN 1 ELSE 0 END) AS weekly_shop_itemClick,

    SUM(CASE 
        WHEN cntxt_nm = 'browseShop' AND page_nm = 'browseShop' AND overlay_nm IS NULL
        THEN 1 ELSE 0 END) AS browse_shop_itemClick,

    SUM(CASE 
        WHEN cntxt_nm = 'categoryShop' AND page_nm = 'categoryShop' AND overlay_nm IS NULL
        THEN 1 ELSE 0 END) AS category_shop_itemClick,

    SUM(CASE 
        WHEN ( (cntxt_nm = 'sparkyAssistant' AND page_nm = 'productPage') 
               OR (overlay_nm = 'sparkyChatWindow') )
        THEN 1 ELSE 0 END) AS carousel_itemClick
FROM item_clicks
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
item_clicks = pd.DataFrame(rows, columns=cols)
item_clicks.head()

Item Clicks Grouped by Context Category


,event_shop_itemClick,weekly_shop_itemClick,browse_shop_itemClick,category_shop_itemClick,carousel_itemClick
0,113,596,72,2560,6201


## Customer Usage - Shop Engagement Volume - Messages

In [99]:
print("Messages grouped by Context Category")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),

messages AS (
    SELECT  
        event_dt, 
        a.pg_cust_id, 
        cntxt_nm,
        brwsr_sessn_cooke_val,
        actn_sub_catg_nm,
        json_extract_scalar(page_attr, '$.nm') AS page_nm,
        json_extract_scalar(payload_txt, '$.pageViewId') as pageViewId
    FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw a
    JOIN invited_users i
      ON i.pg_cust_id = a.pg_cust_id
    WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
      AND (assoc_ind = 'false' OR assoc_ind IS NULL)
      AND json_extract_scalar(payload_txt, '$.buttonName') = 'sendSparkyChat'
    GROUP BY 1,2,3,4,5,6,7
)

SELECT
    SUM(CASE 
        WHEN cntxt_nm = 'eventShop' AND page_nm = 'eventShop'
        THEN 1 ELSE 0 END) AS event_shop_msg,

    SUM(CASE 
        WHEN cntxt_nm = 'weeklyShop' AND page_nm = 'weeklyShop'
        THEN 1 ELSE 0 END) AS weekly_shop_msg,

    SUM(CASE 
        WHEN cntxt_nm = 'browseShop' AND page_nm = 'browseShop'
        THEN 1 ELSE 0 END) AS browse_shop_msg,

    SUM(CASE 
        WHEN cntxt_nm = 'categoryShop' AND page_nm = 'categoryShop'
        THEN 1 ELSE 0 END) AS category_shop_msg,

    SUM(CASE 
        WHEN cntxt_nm = 'homePage' AND page_nm = 'homePage'
        THEN 1 ELSE 0 END) AS home_page_msg,

    SUM(CASE 
        WHEN cntxt_nm = 'productPage' AND page_nm = 'productPage'
        THEN 1 ELSE 0 END) AS product_page_msg

FROM messages
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
messages_by_shop = pd.DataFrame(rows, columns=cols)
messages_by_shop

Messages grouped by Context Category


,event_shop_msg,weekly_shop_msg,browse_shop_msg,category_shop_msg,home_page_msg,product_page_msg
0,24,211,33,3301,12854,543


## Business Enablement - Total Orders (Both Classic and Raptor App) by Logged In Users

In [100]:
print("Total Order Count for Logged-In Users")

sql = f"""
WITH invited_users AS (
    SELECT DISTINCT pg_cust_acct_id AS pg_cust_id
    FROM gpadata_analytics.{invited_users}
),
login_customers AS (
    SELECT DISTINCT a.pg_cust_id AS login_users
    FROM (
        SELECT DISTINCT pg_cust_id, visitor_cooke_val, brwsr_sessn_cooke_val
        FROM ww_csd_dl_tables.csd_site_traffic_rptr_event_log_vw
        WHERE event_dt BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
          AND (assoc_ind = 'false' OR assoc_ind IS NULL)
          AND json_extract_scalar(page_attr, '$.nm') = 'homePage'
          AND actn_catg_nm = 'pageView'
    ) a
    JOIN invited_users i
      ON i.pg_cust_id = a.pg_cust_id
)
SELECT COUNT(DISTINCT order_nbr) AS total_order_count_bothApps
FROM ww_crew_dl_rpt_vm.cnsld_order_item
WHERE
    CAST(order_plcd_ts_utc AS DATE) BETWEEN DATE '{start_dt}' AND DATE '{end_dt}'
    AND order_plcd_dt BETWEEN DATE '{start_dt}' - INTERVAL '1' DAY AND DATE '{end_dt}' + INTERVAL '1' DAY
    AND op_cmpny_cd = 'WMT.COM'
    AND tenant_ste_nm IN ('US_GLASS_IPHONE', 'US_GLASS_ANDROID')
    AND cust_id IN (
        SELECT DISTINCT CAST(login_users AS VARCHAR) FROM login_customers
    )
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]
order_count_both_apps = pd.DataFrame(rows, columns=cols)
order_count_both_apps.head()

Total Order Count for Logged-In Users


,total_order_count_bothApps
0,4706


In [ ]:
# Merge side by side
df_final = pd.concat([glass_app_session, login_users, interacted_users, bounce_users, meaningful_interacted_users, meaningful_interacted_sessn, meaningful_interected_event_counts, 
                      opened_shop_cust, opened_shop_sessn,count_opened_shop, created_shop, created_shop_type, atc_users, checkout_users,
                      checkout_sessn, order_users, order_sessn, repeated_order_users, inchat_bounce_users, inchat_msg, session_duration, users_w_carousel_imp, pdp_imp, atc_context_pivot, item_clicks, messages_by_shop, order_count_both_apps], axis=1)

df_final

,classic_users,classic_sessn,login_users,login_sessn,interacted_users,interacted_sessn,bounce_cust_ct,bounce_sessn_ct,users_meaningful_interaction,users_sent_msg,...,browse_shop_itemClick,category_shop_itemClick,carousel_itemClick,event_shop_msg,weekly_shop_msg,browse_shop_msg,category_shop_msg,home_page_msg,product_page_msg,total_order_count_bothApps
0,12240,108805,6085,17965,5901,16634,39,41,5247,4497,...,72,2560,6201,24,211,33,3301,12854,543,4706


In [103]:
df_final.to_clipboard(index=True)

